In [1]:
import os
import sys
sys.path.append('..')
os.environ['TF_XLA_FLAGS'] = "--tf_xla_auto_jit=fusible"
import tensorflow as tf
from tensorflow.keras.mixed_precision import experimental as mixed_precision

import horovod.tensorflow as hvd
hvd.init()

physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[hvd.rank()], True)
tf.config.set_visible_devices(physical_devices[hvd.rank()], 'GPU')
devices = tf.config.list_logical_devices('GPU')

tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)

In [2]:
from tqdm import tqdm
from statistics import mean
#import GPUtil
from mask_rcnn.tf2_model import MaskRCNN
from mask_rcnn.hyperparameters import dataset_params

In [9]:
from mask_rcnn.tf2_model import MaskRCNN
from mask_rcnn.hyperparameters import dataset_params
from mask_rcnn.hyperparameters import mask_rcnn_params
from mask_rcnn import dataset_utils
from mask_rcnn.training import losses, learning_rates
from simple_model.tf2 import weight_loader, train, scheduler
from simple_model import model_v2
from evaluation import compute_coco_eval_metric_nonestimator, process_prediction_for_eval

train_file_pattern = '/home/ubuntu/tfr_anchor/train*'
eval_file_pattern = '/home/ubuntu/tfr_anchor/val*'
batch_size = 4
global_batch_size = batch_size * hvd.size()
images = 118287
steps_per_epoch = images//global_batch_size
train_data_params = dataset_params.get_data_params()
eval_data_params = dataset_params.get_data_params()
params = mask_rcnn_params.default_config().values()
train_data_params['batch_size'] = batch_size
eval_data_params['batch_size'] = 1
params['finetune_bn'] = False
params['train_batch_size'] = batch_size
params['l2_weight_decay'] = 1e-4
params['init_learning_rate'] = 2e-3 * global_batch_size
params['warmup_learning_rate'] = 2e-4 * global_batch_size
params['warmup_steps'] = 4096//global_batch_size
params['learning_rate_steps'] = [steps_per_epoch * 9, steps_per_epoch * 11]
params['learning_rate_levels'] = [2e-4 * global_batch_size, 2e-5 * global_batch_size]
params['momentum'] = 0.9
params['use_batched_nms'] = False
params['use_custom_box_proposals_op'] = True
params['amp'] = True
params['include_groundtruth_in_features'] = True

In [10]:
eval_data_params = dataset_params.get_data_params()
eval_data_params['batch_size'] = 1

eval_loader = dataset_utils.FastDataLoader(eval_file_pattern, eval_data_params)
eval_tdf = eval_loader(eval_data_params)
eval_tdf = eval_tdf.apply(tf.data.experimental.prefetch_to_device(devices[0].name, 
                                                                    buffer_size=tf.data.experimental.AUTOTUNE))
eval_iter = iter(eval_tdf)

mask_rcnn = model_v2.MRCNN(params)


[MaskRCNN] INFO    : Using Dataset Sharding with Horovod


In [ ]:
mask_rcnn.sa

In [8]:
mask_rcnn.load_weights()
@tf.function
def pred(features, params):
    out = mask_rcnn(features, None, params, is_training=False)
    out['image_info'] = features['image_info']
    out['source_id'] = features['source_ids']
    return out

TypeError: load_weights() missing 1 required positional argument: 'filepath'

In [ ]:
val_json_file="/home/ubuntu/nv_tfrecords/annotations/instances_val2017.json"
eval_steps = 5000
progressbar_eval = tqdm(range(eval_steps))
worker_predictions = dict()    
if hvd.rank()==0:
    print("Beginning eval")
    progressbar_eval = tqdm(range(eval_steps))
else:
    progressbar_eval = range(eval_steps)

for i in progressbar_eval:
    features_val, _ = next(val_iter)
    out = pred(features_val, params)
    out = process_prediction_for_eval(out)

    for k, v in out.items():
        if k not in worker_predictions:
            worker_predictions[k] = [v]
        else:
            worker_predictions[k].append(v)

compute_coco_eval_metric_nonestimator(worker_predictions, annotation_json_file=val_json_file)